In [3]:
import pandas as pd
import numpy as np

In [4]:
FAST, SLOW = 5, 15
data = pd.read_csv('SP500 stock OHLC extended.csv', header=0, index_col=0, parse_dates=[0])
data

,Open,High,Low,Close,Adj Close,Volume,symbol,industry
Date,,,,,,,,
2003-06-02,13.590844,13.683834,13.268956,13.397711,11.463510,6300926.0,A,Health Care
2003-06-03,13.397711,13.648069,13.319027,13.626609,11.659362,4041758.0,A,Health Care
2003-06-04,13.590844,14.270386,13.583691,14.241774,12.185717,2874568.0,A,Health Care
2003-06-05,14.127325,14.163090,13.762518,14.055794,12.026588,4139758.0,A,Health Care
2003-06-06,14.127325,14.234621,13.733906,13.812589,11.818497,4395452.0,A,Health Care
...,...,...,...,...,...,...,...,...
2022-10-24,48.320000,50.049999,48.130001,49.779999,49.377518,2713000.0,ZION,Financials
2022-10-25,47.720001,51.869999,47.490002,51.509998,51.093529,3797100.0,ZION,Financials
2022-10-26,51.040001,51.939999,50.349998,50.400002,49.992508,1623000.0,ZION,Financials


In [5]:
def SMA_ratio(data, fast, slow):
    data['SMA_fast'] = data.groupby('symbol').Close.transform(lambda x: x.rolling(window=fast).mean())
    data['SMA_slow'] = data.groupby('symbol').Close.transform(lambda x: x.rolling(window=slow).mean())
    data['SMA_ratio'] = data.SMA_fast/data.SMA_slow

In [6]:
def SMA_volume_ratio(data, fast, slow):
    data['SMA_fast_volume'] = data.groupby('symbol').Volume.transform(lambda x: x.rolling(window=fast).mean())
    data['SMA_slow_volume'] = data.groupby('symbol').Volume.transform(lambda x: x.rolling(window=slow).mean())
    data['SMA_volume_ratio'] = data.SMA_fast_volume/data.SMA_slow_volume

In [7]:
def Wilder(data, period):
    # Wilder's Smoothing
    start = np.where(~np.isnan(data))[0][0]
    wilder = np.array([np.nan] * len(data))
    wilder[start+period-1] = data[start:(start+period)].mean()
    for i in range(start+period, len(data)):
        wilder[i] = (wilder[i-1] * (period-1) + data[i])/period
    return wilder

In [8]:
def ATR(data, fast, slow):
    # Average True Range
    data['prev_close'] = data.groupby('symbol').Close.shift(1)
    data['TR'] = np.maximum((data.High - data.Low),
                np.maximum(abs(data.High - data.prev_close),
                abs(data.prev_close - data.Close)))
    for ticker in data.symbol.unique():
        temp = data[data.symbol == ticker].copy()
        data.loc[data.symbol == ticker, 'ATR_fast'] = Wilder(temp['TR'], fast)
        data.loc[data.symbol == ticker, 'ATR_slow'] = Wilder(temp['TR'], slow)
    data['ATR_ratio'] = data.ATR_fast/data.ATR_slow

In [9]:
def ADX(data, fast, slow):
    # Average Directional Index
    data['prev_high'] = data.groupby('symbol').High.shift(1)
    data['prev_low'] = data.groupby('symbol').Low.shift(1)
    data['+DM'] = np.where(~np.isnan(data.prev_high),
                           np.where((data['High'] > data['prev_high']) & 
         (((data['High'] - data['prev_high']) > (data['prev_low'] - data['Low']))), 
                                                                  data['High'] - data['prev_high'], 
                                                                  0),np.nan)

    data['-DM'] = np.where(~np.isnan(data.prev_low),
                               np.where((data['prev_low'] > data['Low']) & 
             (((data['prev_low'] - data['Low']) > (data['High'] - data['prev_high']))), 
                                        data['prev_low'] - data['Low'], 
                                        0),np.nan)

    for ticker in data['symbol'].unique():
        temp = data[data.symbol == ticker].copy()
        data.loc[data.symbol==ticker,'+DM_fast'] = Wilder(temp['+DM'], fast)
        data.loc[data.symbol==ticker,'-DM_fast'] = Wilder(temp['-DM'], fast)
        data.loc[data.symbol==ticker,'+DM_slow'] = Wilder(temp['+DM'], slow)
        data.loc[data.symbol==ticker,'-DM_slow'] = Wilder(temp['-DM'], slow)

    data['+DI_fast'] = (data['+DM_fast']/data['ATR_fast'])*100
    data['-DI_fast'] = (data['-DM_fast']/data['ATR_fast'])*100
    data['+DI_slow'] = (data['+DM_slow']/data['ATR_slow'])*100
    data['-DI_slow'] = (data['-DM_slow']/data['ATR_slow'])*100

    data['DX_fast'] = (np.round(abs(data['+DI_fast'] - data['-DI_fast'])/(data['+DI_fast'] + data['-DI_fast']) * 100))

    data['DX_slow'] = (np.round(abs(data['+DI_slow'] - data['-DI_slow'])/(data['+DI_slow'] + data['-DI_slow']) * 100))

    for ticker in data['symbol'].unique():
        temp = data[data.symbol == ticker].copy()
        data.loc[data.symbol==ticker,'ADX_fast'] = Wilder(temp['DX_fast'], fast)
        data.loc[data.symbol==ticker,'ADX_slow'] = Wilder(temp['DX_slow'], slow)

In [10]:
def StochOsc(data, fast, slow):
    data['Lowest_fast'] = data.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = fast).min())
    data['High_fast'] = data.groupby('symbol')['High'].transform(lambda x: x.rolling(window = fast).max())
    data['Lowest_slow'] = data.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = slow).min())
    data['High_slow'] = data.groupby('symbol')['High'].transform(lambda x: x.rolling(window = slow).max())

    data['Stochastic_fast'] = ((data['Close'] - data['Lowest_fast'])/(data['High_fast'] - data['Lowest_fast']))*100
    data['Stochastic_slow'] = ((data['Close'] - data['Lowest_slow'])/(data['High_slow'] - data['Lowest_slow']))*100

    data['Stochastic_%D_fast'] = data['Stochastic_fast'].rolling(window = fast).mean()
    data['Stochastic_%D_slow'] = data['Stochastic_slow'].rolling(window = slow).mean()

    data['Stochastic_Ratio'] = data['Stochastic_%D_fast']/data['Stochastic_%D_slow']

In [11]:
def RSI(data, fast, slow):
    data['Diff'] = data.groupby('symbol')['Close'].transform(lambda x: x.diff())
    data['Up'] = data['Diff']
    data.loc[(data['Up']<0), 'Up'] = 0

    data['Down'] = data['Diff']
    data.loc[(data['Down']>0), 'Down'] = 0 
    data['Down'] = abs(data['Down'])

    data['avg_fast_up'] = data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=fast).mean())
    data['avg_fast_down'] = data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=fast).mean())

    data['avg_slow_up'] = data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=slow).mean())
    data['avg_slow_down'] = data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=slow).mean())

    data['RS_fast'] = data['avg_fast_up'] / data['avg_fast_down']
    data['RS_slow'] = data['avg_slow_up'] / data['avg_slow_down']

    data['RSI_fast'] = 100 - (100/(1+data['RS_fast']))
    data['RSI_slow'] = 100 - (100/(1+data['RS_slow']))

    data['RSI_ratio'] = data['RSI_fast']/data['RSI_slow']

In [12]:
def MACD(data, fast, slow):
    data['fast_Ewm'] = data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span=fast, adjust=False).mean())
    data['slow_Ewm'] = data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span=slow, adjust=False).mean())
    data['MACD'] = data['fast_Ewm'] - data['slow_Ewm']

In [13]:
def RateofChange(data, slow):
    data['RC'] = data.groupby('symbol')['Close'].transform(lambda x: x.pct_change(periods = slow))

In [14]:
def get_indicators(data, fast, slow):
    SMA_ratio(data, fast, slow)
    SMA_volume_ratio(data, fast, slow)
    ATR(data, fast, slow)
    ADX(data, fast, slow)
    StochOsc(data, fast, slow)
    RSI(data, fast, slow)
    MACD(data, fast, slow)
    RateofChange(data, slow)

In [15]:
get_indicators(data, FAST, SLOW)
data.to_csv("SP500 stock TI extended.csv")
print(data)

                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2003-06-02  13.590844  13.683834  13.268956  13.397711  11.463510  6300926.0   
2003-06-03  13.397711  13.648069  13.319027  13.626609  11.659362  4041758.0   
2003-06-04  13.590844  14.270386  13.583691  14.241774  12.185717  2874568.0   
2003-06-05  14.127325  14.163090  13.762518  14.055794  12.026588  4139758.0   
2003-06-06  14.127325  14.234621  13.733906  13.812589  11.818497  4395452.0   
...               ...        ...        ...        ...        ...        ...   
2022-10-24  48.320000  50.049999  48.130001  49.779999  49.377518  2713000.0   
2022-10-25  47.720001  51.869999  47.490002  51.509998  51.093529  3797100.0   
2022-10-26  51.040001  51.939999  50.349998  50.400002  49.992508  1623000.0   
2022-10-27  51.060001  51.520000  50.189999  50.380001  49.972668  1134600.0   
2022-10-28  50.779999  51.610001  50.430